In [0]:
from pyspark.sql import functions as F, types as T
import json

dbutils.widgets.text("dt", "")
dbutils.widgets.text("bucket", "")
dbutils.widgets.text("bronze_prefix", "")
dbutils.widgets.text("silver_prefix", "")
dbutils.widgets.dropdown("table", "chain_daily_state", ["chain_daily_state", "active_locks_daily"])

debug: bool = False






In [0]:

DEC_UOPT = T.DecimalType(38, 0)

CHAIN_DAILY_SCHEMA = T.StructType([
    T.StructField("dt", T.StringType(), False),
    T.StructField("height", T.LongType(), True),
    T.StructField("snapshot_ts", T.TimestampType(), True),
    T.StructField("total_supply_uopt", DEC_UOPT, True),
    T.StructField("total_locked_uopt", DEC_UOPT, True),
    T.StructField("bonded_uopt", DEC_UOPT, True),
    T.StructField("not_bonded_uopt", DEC_UOPT, True),
    T.StructField("source_run_ids", T.MapType(T.StringType(), T.StringType()), True),
])


In [0]:
dt = dbutils.widgets.get("dt")
bucket = dbutils.widgets.get("bucket").strip()
bronze_prefix = dbutils.widgets.get("bronze_prefix").strip().rstrip("/")
silver_prefix = dbutils.widgets.get("silver_prefix").strip().rstrip("/")
table = dbutils.widgets.get("table")

if not bucket:
    raise ValueError("bucket is required")

bronze_dt = f"s3://{bucket}/{bronze_prefix}/dt={dt}"
manifest_path = f"{bronze_dt}/_MANIFEST.json"



In [0]:
%sql

DESCRIBE EXTERNAL LOCATION `db_s3_external_databricks-s3-optio-bronze-raw`;


GRANT READ FILES
ON EXTERNAL LOCATION `db_s3_external_databricks-s3-optio-bronze-raw`
TO `optio-devs`;

SHOW GRANTS ON EXTERNAL LOCATION `db_s3_external_databricks-s3-optio-bronze-raw`;

In [0]:
def read_manifest(path: str) -> dict:
    txt = dbutils.fs.head(path, 1000000)
    return json.loads(txt)

manifest = read_manifest(manifest_path)

display(manifest)


In [0]:
def dataset_run_path(dataset: str) -> str:
    info = manifest["datasets"].get(dataset)
    if not info or info.get("status") != "SUCCESS":
        raise ValueError(f"Dataset not SUCCESS in manifest: {dataset}")
    run_id = info["run_id"]
    return f"s3://{bucket}/{bronze_prefix}/dt={dt}/dataset={dataset}/run_id={run_id}"


df = spark.read.json(f"{dataset_run_path('staking_delegations_to')}/part-*.jsonl.gz")
# display(df.limit(5))


In [0]:

def extract_uopt_from_supply_df(df_supply):
    """
    Your observed shape:
    raw_json = {"supply":[{"amount":"...","denom":"..."}, ...], ...}
    Extract the uOPT amount.
    """
    raw_json_schema = T.StructType([
        T.StructField("supply", T.ArrayType(
            T.StructType([
                T.StructField("amount", T.StringType(), True),
                T.StructField("denom", T.StringType(), True),
            ])
        ), True)
    ])

    raw_str = F.to_json(F.col("raw_json"))
    parsed = df_supply.withColumn("raw_parsed", F.from_json(raw_str, raw_json_schema))

    coins = (parsed
        .withColumn("coin", F.explode(F.col("raw_parsed.supply")))
        .select(
            F.col("snapshot_height").cast("bigint").alias("height"),
            F.to_timestamp("extracted_at").alias("snapshot_ts"),
            F.col("coin.denom").alias("denom"),
            F.col("coin.amount").alias("amount_str"),
        )
    )

    uopt = (coins
        .where(F.col("denom") == F.lit("uOPT"))
        .select(
            "height",
            "snapshot_ts",
            F.col("amount_str").cast("string").cast(DEC_UOPT).alias("total_supply_uopt"),
        )
    )

    return uopt


In [0]:
def read_manifest(path: str) -> dict:
    txt = dbutils.fs.head(path, 1000000)
    return json.loads(txt)

def dataset_run_path(bucket: str, bronze_prefix: str, dt: str, manifest: dict, dataset: str) -> str:
    info = manifest["datasets"].get(dataset)
    if not info:
        raise ValueError(f"Dataset missing from manifest: {dataset}")
    if info.get("status") != "SUCCESS":
        raise ValueError(f"Dataset not SUCCESS in manifest: {dataset} => {info}")
    run_id = info["run_id"]
    return f"s3://{bucket}/{bronze_prefix}/dt={dt}/dataset={dataset}/run_id={run_id}"

def read_bronze_jsonl_gz(run_path: str):
    return spark.read.json(f"{run_path}/part-*.jsonl.gz")



In [0]:
manifest_path = f"s3://{bucket}/{bronze_prefix}/dt={dt}/_MANIFEST.json"
manifest = read_manifest(manifest_path)
if debug:
  print("snapshot_height:", manifest["snapshot_height"])
  print("datasets:", list(manifest["datasets"].keys()))

run_path = dataset_run_path(bucket, bronze_prefix, dt, manifest, "staking_delegations_to")
if debug:
  print("run_path:", run_path)

df = read_bronze_jsonl_gz(run_path)
if debug:
  print("rows:", df.count())
  display(df.limit(5))


In [0]:

def as_dec_uopt(col):
    return F.col(col).cast(DEC_UOPT)

def as_ts(col):
    return F.to_timestamp(F.col(col))

def as_date(col):
    return F.to_date(F.col(col))

In [0]:
df2 = (df
  .select(
    F.col("dt"),
    F.col("snapshot_height").cast("bigint").alias("height"),
    as_ts("extracted_at").alias("snapshot_ts"),
    F.col("delegator_address"),
    F.col("validator_operator_address"),
    as_dec_uopt("balance_amount_uopt").alias("balance_amount_uopt_dec"),
    F.col("shares").cast(T.DecimalType(38,18)).alias("shares_dec"),
  )
)

# display(df2.limit(5))

In [0]:
%sql
USE CATALOG optio_warehouse;
USE SCHEMA silver;

CREATE TABLE IF NOT EXISTS optio_warehouse.silver.silver_active_locks_daily (
  dt STRING,
  height BIGINT,
  snapshot_ts TIMESTAMP,
  address STRING,
  unlock_date DATE,
  denom STRING,
  amount_uopt DECIMAL(38,0),
  lock_id STRING,
  source_run_id STRING
)
USING DELTA
PARTITIONED BY (dt);

CREATE TABLE IF NOT EXISTS optio_warehouse.silver.silver_chain_daily_state (
  dt STRING,
  height BIGINT,
  snapshot_ts TIMESTAMP,
  total_supply_uopt DECIMAL(38,0),
  total_locked_uopt DECIMAL(38,0),
  bonded_uopt DECIMAL(38,0),
  not_bonded_uopt DECIMAL(38,0),
  source_run_ids MAP<STRING, STRING>
)
USING DELTA
PARTITIONED BY (dt);


In [0]:
%sql
SHOW TABLES IN optio_warehouse.silver;

In [0]:
DEC_SHARES = T.DecimalType(38, 18)
DEC_UOPT   = T.DecimalType(38, 0)

def as_dec_uopt(col):
    return F.col(col).cast(DEC_UOPT)

def as_ts(col):
    return F.to_timestamp(F.col(col))

def as_date(col):
    return F.to_date(F.col(col))

def build_silver_active_locks_daily(bucket: str, bronze_prefix: str, dt: str, manifest: dict):
    dataset = "lockup_active_locks"
    run_id = manifest["datasets"][dataset]["run_id"]
    run_path = dataset_run_path(bucket, bronze_prefix, dt, manifest, dataset)

    bronze = spark.read.json(f"{run_path}/part-*.jsonl.gz")

    # Create deterministic lock_id from stable business fields
    df = (bronze
        .select(
            F.lit(dt).alias("dt"),
            F.col("snapshot_height").cast("bigint").alias("height"),
            as_ts("extracted_at").alias("snapshot_ts"),
            F.col("address").cast("string").alias("address"),
            as_date("unlock_date").alias("unlock_date"),
            F.col("amount_denom").cast("string").alias("denom"),
            as_dec_uopt("amount_uopt").alias("amount_uopt"),
            F.lit(run_id).alias("source_run_id"),
        )
        .withColumn(
            "lock_id",
            F.sha2(
                F.concat_ws("|",
                    F.col("dt"),
                    F.col("height").cast("string"),
                    F.col("address"),
                    F.col("unlock_date").cast("string"),
                    F.col("denom"),
                    F.col("amount_uopt").cast("string"),
                ),
                256
            )
        )
        .dropDuplicates(["dt", "lock_id"])
    )
    return df


In [0]:
df_locks = build_silver_active_locks_daily(bucket, bronze_prefix, dt, manifest)
if debug:
    print("rows:", df_locks.count())
    print("distinct lock_id:", df_locks.select("lock_id").distinct().count())
    display(df_locks.limit(10))

In [0]:
target_table = "optio_warehouse.silver.silver_active_locks_daily"

(df_locks.write
  .format("delta")
  .mode("overwrite")
  .option("replaceWhere", f"dt = '{dt}'")
  .saveAsTable(target_table))


In [0]:
if debug:
    spark.table(target_table).where(F.col("dt") == dt).count()


In [0]:
def json_get_dec(raw_json_col: str, *json_paths: str):
    """
    Extract the first non-null value among json_paths from raw_json as DECIMAL(38,0).
    raw_json may be struct or string; to_json normalizes.
    """
    raw_str = F.to_json(F.col(raw_json_col))
    candidates = [F.get_json_object(raw_str, p) for p in json_paths]
    return F.coalesce(*[c.cast("string") for c in candidates]).cast(DEC_UOPT)

def json_total_locked_uopt(raw_json_col: str = "raw_json"):
    raw_str = F.to_json(F.col(raw_json_col))
    return F.coalesce(
        # common: {"amount":{"amount":"123","denom":"uOPT"}}
        F.get_json_object(raw_str, "$.amount.amount"),
        # common: {"amount":"123","denom":"uOPT"}
        F.get_json_object(raw_str, "$.amount"),
        # some variants: {"total_locked":{"amount":"123","denom":"uOPT"}}
        F.get_json_object(raw_str, "$.total_locked.amount"),
        # some variants: {"total_locked":"123"}
        F.get_json_object(raw_str, "$.total_locked"),
        # last resort: maybe nested differently
        F.get_json_object(raw_str, "$.coin.amount"),
    ).cast("string").cast(DEC_UOPT)


In [0]:
def build_silver_chain_daily_state(bucket: str, bronze_prefix: str, dt: str, manifest: dict):
    ds_supply = "bank_total_supply"
    ds_locked = "lockup_total_locked"
    ds_pool   = "staking_pool"

    p_supply = dataset_run_path(bucket, bronze_prefix, dt, manifest, ds_supply)
    p_locked = dataset_run_path(bucket, bronze_prefix, dt, manifest, ds_locked)
    p_pool   = dataset_run_path(bucket, bronze_prefix, dt, manifest, ds_pool)

    df_supply = spark.read.json(f"{p_supply}/part-*.jsonl.gz")
    df_locked = spark.read.json(f"{p_locked}/part-*.jsonl.gz")
    df_pool   = spark.read.json(f"{p_pool}/part-*.jsonl.gz")

    # Guardrails
    assert df_supply.count() == 1, "bank_total_supply expected 1 row"
    assert df_locked.count() == 1, "lockup_total_locked expected 1 row"
    assert df_pool.count() == 1, "staking_pool expected 1 row"

    uopt_supply_df = extract_uopt_from_supply_df(df_supply)

    # Guardrail: should find exactly one uOPT row
    cnt = uopt_supply_df.count()
    if cnt != 1:
        raise ValueError(f"bank_total_supply expected exactly 1 uOPT coin row, found {cnt}")

    r_supply = uopt_supply_df.first()

    r_locked = (df_locked
        .select(
            json_total_locked_uopt("raw_json").alias("total_locked_uopt"),
        )
        .first()
    )

    r_pool = (df_pool
        .select(
            json_get_dec("raw_json", "$.bonded_tokens", "$.pool.bonded_tokens").alias("bonded_uopt"),
            json_get_dec("raw_json", "$.not_bonded_tokens", "$.pool.not_bonded_tokens").alias("not_bonded_uopt"),
        )
        .first()
    )

    source_run_ids = {
        ds_supply: manifest["datasets"][ds_supply]["run_id"],
        ds_locked: manifest["datasets"][ds_locked]["run_id"],
        ds_pool: manifest["datasets"][ds_pool]["run_id"],
    }

    # Debug: show extracted values before building DF
    if debug:
        print("r_supply:", dict(r_supply.asDict()))
        print("r_locked:", dict(r_locked.asDict()))
        print("r_pool:", dict(r_pool.asDict()))

    row_dict = {
        "dt": dt,
        "height": int(r_supply["height"]) if r_supply["height"] is not None else None,
        "snapshot_ts": r_supply["snapshot_ts"],
        "total_supply_uopt": r_supply["total_supply_uopt"],
        "total_locked_uopt": r_locked["total_locked_uopt"],
        "bonded_uopt": r_pool["bonded_uopt"],
        "not_bonded_uopt": r_pool["not_bonded_uopt"],
        "source_run_ids": source_run_ids,
    }

    out = spark.createDataFrame([row_dict], schema=CHAIN_DAILY_SCHEMA)

    # Fail fast if any critical numeric came back null
    row = out.first()
    for k in ["total_supply_uopt", "total_locked_uopt", "bonded_uopt", "not_bonded_uopt"]:
        if row[k] is None:
            raise ValueError(f"{k} resolved to null. Check raw_json shapes in bronze datasets for dt={dt}.")

    
    row = out.first()
    assert row["total_supply_uopt"] >= row["bonded_uopt"], "Supply should be >= bonded"
    assert row["total_supply_uopt"] >= row["total_locked_uopt"], "Supply should be >= locked"

    return out



In [0]:
df_chain = build_silver_chain_daily_state(bucket, bronze_prefix, dt, manifest)

if debug:
    display(df_chain)
    print("rows:", df_chain.count())


In [0]:
ds = "staking_pool"
p_pool = dataset_run_path(bucket, bronze_prefix, dt, manifest, ds)
df_pool = spark.read.json(f"{p_pool}/part-*.jsonl.gz")
if debug:
    df_pool.printSchema()
    
display(df_pool.select("raw_json").limit(1))


In [0]:
target_table_chain = "optio_warehouse.silver.silver_chain_daily_state"

(df_chain.write
  .format("delta")
  .mode("overwrite")
  .option("replaceWhere", f"dt = '{dt}'")
  .saveAsTable(target_table_chain))

In [0]:
spark.table(target_table_chain).where(F.col("dt") == dt).count()


In [0]:
%sql
SELECT
  dt, height, snapshot_ts,
  total_supply_uopt, total_locked_uopt,
  bonded_uopt, not_bonded_uopt,
  source_run_ids
FROM optio_warehouse.silver.silver_chain_daily_state
WHERE dt = '2026-01-25';

In [0]:
ds = "bank_total_supply"
p_supply = dataset_run_path(bucket, bronze_prefix, dt, manifest, ds)
df_supply = spark.read.json(f"{p_supply}/part-*.jsonl.gz")

display(df_supply.select("raw_json").limit(1))
